# B-scan from a metal cylinder (2D)

This example uses the same geometry as the previous (A-scan) example but this time a B-scan is created. A B-scan is composed of multiple traces (A-scans) recorded as the source and receiver are moved over the target, in this case the metal cylinder. The input needed to create the model is:

### my_cylinder_Bscan_2D.in

In [ ]:
%%writefile gprMax/user_models/cylinder_Bscan_2D.in
#soil_peplinski: 0.5 0.5 2.0 2.66 0.001 0.25 my_soil
------------------------------------------------------------
#domain: 1.0 1.0 0.1
#dx_dy_dz: 0.002 0.002 0.002
#time_window: 12e-9
------------------------------------------------------------
#fractal_box: 0 0 0 1 0.75 0.1 1.5 1 1 1 50 my_soil my_soil_box 42 y
#cylinder: 0.33 0.33 0 0.33 0.33 0.002 0.01 pec y
-------------------------------------------------------------
-------------------------------------------------------------
#rx: 0.1125 0.1525 0
#src_steps: 0.002 0.0 0
#rx_steps: 0.002 0.0 0
------------------------------------------------------------
#waveform: ricker 1 1.5e9 my_ricker
#hertzian_dipole: z 0.150 0.170 0 my_ricker
#messages: y

Overwriting gprMax/user_models/cylinder_Bscan_2D.in


The differences between this input file and the one from the A-scan are the x coordinates of the source and receiver, and the commands needed to move the source and receiver. As before, the source and receiver are offset by 40mm from each other as before but they are now shifted to a starting position for the scan. The ``#src_steps`` command is used to move every source in the model by specified steps each time the model is run. Similarly, the ``#rx_steps`` command is used to move every receiver in the model by specified steps each time the model is run. Note, the same functionality can be achieved by using a block of Python code in the input file to move the source and receiver individually (for further details see the Python section of the User Guide).

## Run the model

To run the model to create a B-scan you must pass an optional argument to specify the number of times the model should be run. In this case this is the number of A-scans (traces) that will comprise the B-scan. For a B-scan over a distance of 120mm with a step of 2mm that is 60 A-scans. You can now run the model using:
    
    python -m gprMax user_models/cylinder_Bscan_2D.in -n 60

In [ ]:
import os
from gprMax.gprMax import api

filename = os.path.join('gprMax', 'user_models', 'cylinder_Bscan_2D.in')
api(filename, n=200, geometry_only=False, gpu=[0])


=== Electromagnetic modelling software based on the Finite-Difference Time-Domain (FDTD) method 

    www.gprmax.com   __  __
     __ _ _ __  _ __|  \/  | __ ___  __
    / _` | '_ \| '__| |\/| |/ _` \ \/ /
   | (_| | |_) | |  | |  | | (_| |>  <
    \__, | .__/|_|  |_|  |_|\__,_/_/\_\
    |___/|_|
                     v3.1.4 (Big Smoke)

 Copyright (C) 2015-2018: The University of Edinburgh
 Authors: Craig Warren and Antonis Giannopoulos

 gprMax is free software: you can redistribute it and/or modify it under the
  terms of the GNU General Public License as published by the Free Software
  Foundation, either version 3 of the License, or (at your option) any later
  version.
 gprMax is distributed in the hope that it will be useful, but WITHOUT ANY
  WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
  A PARTICULAR PURPOSE.  See the GNU General Public License for more details.
 You should have received a copy of the GNU General Public License along with
  gpr

## View the results

### Merge A-scans into B-scan

You should have produced 60 output files, one for each A-scan, with names ``my_cylinder_Bscan_2D1.out``, ``my_cylinder_Bscan_2D2.out`` etc... These can be combined into a single file using the command:

    python -m tools.outputfiles_merge user_models/cylinder_Bscan_2D

In [ ]:
from tools.outputfiles_merge import merge_files
filename = os.path.join('gprMax', 'user_models', 'cylinder_Bscan_2D')
merge_files(filename, removefiles=True)

You should see a combined output file ``cylinder_Bscan_2D_merged.out``. The tool will ask you if you want to delete the original single A-scan output files or keep them.

### Plot the B-scan

You can plot the B-scan using:

    python -m tools.plot_Bscan user_models/cylinder_Bscan_2D_merged.out Ez

In [ ]:
%matplotlib inline
import os
from tools.plot_Bscan import get_output_data, mpl_plot
import numpy as np
from utils import save_img

filename = os.path.join('gprMax','user_models', 'cylinder_Bscan_2D_merged.out')
rxnumber = 1
rxcomponent = 'Ez'
outputdata, dt = get_output_data(filename, rxnumber, rxcomponent)
outputdata = outputdata - np.mean(outputdata, axis=1, keepdims=True)
outputdata = outputdata * (
    np.arange(outputdata.shape[0])/outputdata.shape[0]).reshape(outputdata.shape[0], 1)
save_img(filename, outputdata, dt, rxnumber, rxcomponent, 'output-files')

plot = mpl_plot(filename, outputdata, dt, rxnumber, rxcomponent)

The B-scan (of the $E_z$ field component) shows the initial part of the signal (~0.5-1.5 ns) which represents the direct wave from transmitter to receiver. Then comes the refelected wave (~2-3 ns) from the metal cylinder which creates the hyperbolic shape.

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt

inputs = outputdata[:, 32]


print(inputs.shape)
f, t, Zxx = signal.stft(inputs, window='hann', nfft=1024, nperseg=64)

f = f*1000
plt.figure(figsize=(10, 5)).patch.set_facecolor('white')
plt.pcolormesh(f, t, np.abs(Zxx).transpose(), vmin=0)
plt.title("STFT of A-scan")
plt.ylim(t.max(), 0)
plt.xlim(0, f.max())
plt.ylabel('Time (ns)')
plt.xlabel('Frequency (MHz)')
plt.set_cmap('viridis')
plt.colorbar().set_label('Magnitude')

In [ ]:
plt.plot(inputs)

In [ ]:
def stft(inputs, index):
    f, t, zxx = signal.stft(inputs[:, index], window='hann', nperseg=4, nfft=2048, padded=None, boundary=None)
    return f, t, zxx

spects = []
freqs = []
times = []
for i in range(len(outputdata[0,:])):
    f, t, zxx = stft(outputdata, i)
    freqs.append(f)
    times.append(t)
    spects.append(np.abs(zxx))
    
spects = np.array(spects)
freqs = np.array(freqs)
times = np.array(times)
spects = np.amax(spects, axis=1)
print(spects.shape)
print(freqs.shape)
print(times.shape)

print(freqs.max())
plt.figure(figsize=(10,5)).patch.set_facecolor('white')
plt.xlim(0, len(spects[:,0]))
plt.ylim(len(spects[0,:]), 0)
plt.ylabel('Time (ns)')
plt.xlabel('Trace Number')
plt.pcolormesh(spects.transpose()).set_label('Frequency (MHz)')
plt.title('Frequency Distribution of B-scan')
plt.set_cmap('viridis')
plt.tight_layout()
plt.colorbar().set_label('Frequency (MHz)')
plt.savefig('freqency_dist_bscan.png')

In [ ]:
spects.reshape(2, -1).shape

In [ ]:
plt.pcolormesh(freqs.transpose())